In [3]:
import os
folder = 'data/wildtrack'

subfolder_instances_masks = os.path.join(folder, 'output-smpl-3d/instance')
subfolder_output = os.path.join(folder, 'output-keypoints3d/label')

pids = [0,1,2,3,7,9,10]
# join the masks together and invert them


import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from tqdm import tqdm

def load_mask(path):
    return cv2.imread(path, cv2.IMREAD_GRAYSCALE)

def save_mask(path, mask):
    cv2.imwrite(path, mask)
    
def invert_mask(mask):
    return cv2.bitwise_not(mask)

def join_masks(masks):  
    mask = np.zeros_like(masks[0])
    for m in masks:
        mask = cv2.bitwise_or(mask, m)
    return mask

for view_id in tqdm(os.listdir(subfolder_instances_masks)):
    view_folder = os.path.join(subfolder_instances_masks, view_id)
    out_folder = os.path.join(subfolder_output, view_id, "bg")
    os.makedirs(out_folder, exist_ok=True)
    
    masks_per_frame = {}    
    for frame_pid in os.listdir(view_folder):
        # j = '000000_0.png' 'frame_id_pid.png'
        # join the masks with same frame_id
        frame_id = frame_pid.split('_')[0]
        masks_per_frame[frame_id] = masks_per_frame.get(frame_id, []) + [frame_pid]
        
    for frame_id, masks in masks_per_frame.items():
        masks = [load_mask(os.path.join(view_folder, mask)) for mask in masks]
        mask = join_masks(masks)
        mask = invert_mask(mask)
        
        save_mask(os.path.join(out_folder, frame_id + '.png'), mask)

100%|██████████| 7/7 [01:54<00:00, 16.38s/it]
